In [1]:
import json
import numpy as np
import pandas as pd
from collections import Counter

from nltk import word_tokenize, sent_tokenize

from tqdm.auto import tqdm
tqdm.pandas()

pd.set_option('display.max_colwidth', 200)

In [2]:
with open("../data/href2synopsis.json", encoding="utf-8") as f:
    href2synopsis = json.load(f)

    Takeaways:
        - replace \n or \r sequences with single \n
        - use language-specific tokenizer
        - when using max_count_token_ratio make sure do delete stopwords
            (unless you specifically wish to locate examples with too much stopwords)
        - 

# BOW stats
    # TODO: add actual bow stats

In [3]:
data = pd.DataFrame()

data["href"] = href2synopsis.keys()
data["synopsis"] = href2synopsis.values()

In [4]:
data = data[data["synopsis"] != ""]

In [5]:
import regex as re

data["synopsis"] = data["synopsis"].apply(
    lambda x: re.sub("\s*\n\s*", "\n", x).strip()
)

In [75]:
# data["min_ord"] = data["synopsis"].progress_apply(
#     lambda s: min([ord(char) for char in s])
# )

# data["max_ord"] = data["synopsis"].progress_apply(
#     lambda s: max([ord(char) for char in s])
# )

# data["mean_ord"] = data["synopsis"].progress_apply(
#     lambda s: np.mean([ord(char) for char in s])
# )

# data["median_ord"] = data["synopsis"].progress_apply(
#     lambda s: np.median([ord(char) for char in s])
# )

# data["mode_ord"] = data["synopsis"].progress_apply(
#     lambda s: Counter([ord(char) for char in s]).most_common()[0][0]
# )

In [7]:
# data["mode_count"] = data["synopsis"].progress_apply(
#     lambda s: Counter(s).most_common()[0][1]
# )

# data["mode_ratio"] = data["synopsis"].progress_apply(
#     lambda s: Counter(s).most_common()[0][1]/len(s)
# )

# data["mean_char_count"] = data["synopsis"].progress_apply(
#     lambda s: np.mean(list(Counter(s).values()))
# )

# data["mean_token_count"] = data["synopsis"].progress_apply(
#     lambda s: np.mean(list(Counter([w.lower() for w in word_tokenize(s)]).values()))
# )

# data["max_count_token_ratio"] = data["synopsis"].progress_apply(
#     lambda s: Counter([w.lower() for w in word_tokenize(s)]).most_common()[0][1]/len(word_tokenize(s))
# )

In [8]:
from stopwordsiso import stopwords

In [9]:
def max_count_token_ratio_no_stops(text, stopwords=stopwords("en")):
    words = word_tokenize(text)
    counter = Counter()
    for w in words:
        lowered = w.lower()
        if lowered not in stopwords:
            counter.update([lowered])
    if not counter:
        return 0.
    return counter.most_common()[0][1]/len(words)

# data["max_count_token_ratio_stops"] = data["synopsis"].progress_apply(max_count_token_ratio_stops)
data["max_count_token_ratio_no_stops"] = data["synopsis"].progress_apply(max_count_token_ratio_no_stops)

  0%|          | 0/18374 [00:00<?, ?it/s]

In [12]:
data["char_total"] = data["synopsis"].progress_apply(len)
data["token_total"] = data["synopsis"].progress_apply(
    lambda x: len(word_tokenize(x))
)
data["sent_total"] = data["synopsis"].progress_apply(
    lambda x: len([sent for line in x.split("\n") for sent in  sent_tokenize(line) if line.strip()])
)

  0%|          | 0/18374 [00:00<?, ?it/s]

  0%|          | 0/18374 [00:00<?, ?it/s]

  0%|          | 0/18374 [00:00<?, ?it/s]

In [27]:
# data["lines_total"] = data["synopsis"].progress_apply(lambda x: len(x.split("\n")))

  0%|          | 0/19393 [00:00<?, ?it/s]

In [5]:
data["token_max_char_len"] = data["synopsis"].progress_apply(
    lambda x: max([len(t) for t in word_tokenize(x)]) if x.strip() else 0
)

data["sent_max_char_len"] = data["synopsis"].progress_apply(
    lambda x: max([len(t) for t in sent_tokenize(x)]) if x.strip() else 0
)

data["line_max_char_len"] = data["synopsis"].progress_apply(
    lambda x: max([len(t) for t in x.split("\n")]) if x.strip() else 0
)

  0%|          | 0/19393 [00:00<?, ?it/s]

  0%|          | 0/19393 [00:00<?, ?it/s]

  0%|          | 0/19393 [00:00<?, ?it/s]

In [16]:
data["sent_char_ratio"] = [t.sent_total/t.char_total for t in data.itertuples()]
data["sent_token_ratio"] = [t.sent_total/t.token_total for t in data.itertuples()]
data["token_char_ratio"] = [t.token_total/t.char_total for t in data.itertuples()]
data["token_sent_ratio"] = [t.token_total/t.sent_total for t in data.itertuples()]
data["char_sent_ratio"] = [t.char_total/t.sent_total for t in data.itertuples()]
data["char_token_ratio"] = [t.char_total/t.token_total for t in data.itertuples()]

In [10]:
data.describe().astype("float16")

,max_count_token_ratio_no_stops
count,18368.000000
mean,0.058807
std,0.034637
min,0.000000
25%,0.044434
50%,0.054047
75%,0.066650
max,1.000000


In [11]:
stats = data.drop(columns=["href", "synopsis"]) #.drop(columns=["line_max_char_len"])

# Outlier detection

[The svm.OneClassSVM is known to be sensitive to outliers and thus does not perform very well for outlier detection](https://scikit-learn.org/stable/modules/outlier_detection.html#outlier-detection)

In [12]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

In [13]:
clf = LocalOutlierFactor(n_neighbors=20)
preds = clf.fit_predict(stats)

data["pred"] = preds
data["negative_outlier_factor_"] = clf.negative_outlier_factor_

In [14]:
# data[data["pred"] == -1]["synopsis"].sample(10).to_list()

In [15]:
data[data["pred"] == -1].sort_values(by="negative_outlier_factor_"
                                    )[['href', 'synopsis', 'pred', 'negative_outlier_factor_']].head(50)

,href,synopsis,pred,negative_outlier_factor_
15964,/13390-sweet-20,Mini beauty drama.,-1,-2.826929e+08
16506,/729713-enter-the-girl-dragon,~~ Co-prodcution with India,-1,-2.826929e+08
17564,/705993-my-atlantis-boyfriend,~~Aired on Tik Tok,-1,-2.826929e+08
14405,/691835-mr-queen-the-story,Mr. Queen: Commentary,-1,-2.826929e+08
11438,/65247-nct-dream-a-mid-summer-day-s-dream,NCT DREAM A Mid Summer Day's Dream,-1,-2.826929e+08
336,/688491-show-me-the-money-season-10,New Season,-1,-2.246155e+08
2276,/60733-kyou-kara-ore-wa-sp,Drama special to Kyou Kara Ore wa!!,-1,-1.792118e+08
17165,/25378-innate-differences,BL Movie.,-1,-1.769232e+08
1239,/730445-all-of-us-are-dead-season-2,New season.,-1,-1.769232e+08
1338,/703415-new-journey-to-the-west-season-9,New season.,-1,-1.769232e+08


In [16]:
# data.assign(emperor_ratio=data["synopsis"].progress_apply(
#     lambda s: len(re.findall("emperor", s.lower()))/len(word_tokenize(s))
# )).sort_values(by="emperor_ratio", ascending=False).head(10)["synopsis"].to_list()